<a href="https://colab.research.google.com/github/mostume222/inf-test/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print("hello there infa main")

hello there infa main


In [3]:
print("second stuff")

second stuff
